In [ ]:
import random
def coder(n,k,secret): #avec k le nombre de points nécessaires pour décoder et n le nombre de points partagés
    if k <= 0:
        print("Erreur, cette valeur n'est pas valable")
    else :
        f= [secret]
        pts = []
        ptX = 0
        used_ptX = [0]
        for i in range(k-1):
            f.append(random.randint(-30,30))
        for i in range(n):
            
            while ptX in used_ptX:
                ptX = random.randint(-n*3,n*3)
                ptY = 0
                degre = 0
                for i in f:
                    ptY = ptY + i*(ptX**degre)
                    degre += 1
            
            used_ptX.append(ptX)
            pts.append("("+str(ptX)+";"+str(ptY)+")")
        print("Voici les différents points à distribuer à chaques personnes :")
        fichier = open("/home/veriqloud/Documents/Shamir/data.txt", "w")
        for i in pts:
            print(i)
            fichier.write(i+"\n")


In [ ]:
def decoder(k):
    fichier = open("/home/veriqloud/Documents/Shamir/data.txt", "r")
    a = fichier.read()
    fichier.close()
    pts = []
    coordonnes = []
    for i in a :
        coordonnes.append(i)
    v = 0
    signe = 1
    for i in coordonnes:
        if i in ["1","2","3","4","5","6","7","8","9","0"]:
            v = v*10 + int(i)
        elif i == "-":
            signe = -1
        elif i == ";" or i == ")":
            v = v * signe
            pts.append(v)
            signe = 1
            v = 0
    #points enregistrées



    if k <= 0:
        return "Erreur, une des valeurs n'est pas valable"
    else:
        if len(pts)-2*k < 0:
            return "Les points fournis ne sont pas suffisants"
        for i in range(len(pts)-2*k):
            del pts[0]
        Matrice_Gauss = []
        for c in range(k):
            for l in range(k):
                Matrice_Gauss.append(pts[2*c]**l)
            Matrice_Gauss.append(pts[2*c+1])


        impossibilite = 0
        for colonneactive in range(k):
            
            pivot=Matrice_Gauss[colonneactive*(k+1)+colonneactive]
            if pivot == 0:
                impossibilite = 1
                break
            for l in range(k+1):
                
                Matrice_Gauss[colonneactive*(k+1)+l] = Matrice_Gauss[colonneactive*(k+1)+l]/pivot

            for c in range(k):
                coefcolonne = Matrice_Gauss[c*(k+1)+colonneactive]
                for l in range(k+1):
                    if c != colonneactive:
                        Matrice_Gauss[c*(k+1)+l]=Matrice_Gauss[c*(k+1)+l]-Matrice_Gauss[colonneactive*(k+1)+l]*coefcolonne
        
        if impossibilite == 1:
            return "Les points fournis ne sont pas suffisants"
        else:
            print( "Avec les points :\n",a,"\n Le secret est : ",Matrice_Gauss[k], sep = "")

            
        


In [ ]:
decoder(3)

In [ ]:
coder(5,3,8)
